# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
# Import necessary modules
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Loading and reading the data
data = pd.read_csv('homepage_actions.csv')
data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


### Checking the data

In [3]:
data.shape

(8188, 4)

In [4]:
data.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


### Investigating the id column

In [6]:
data.action.unique()

array(['view', 'click'], dtype=object)

In [7]:
data.group.unique()

array(['experiment', 'control'], dtype=object)

In [8]:
experiment_data = data[data.group == 'experiment']

In [9]:
control_data = data[data.group == 'control']

In [10]:
len(experiment_data)

3924

In [11]:
len(control_data)

4264

### Is there any overlap between the control and experiment groups

In [12]:
# merging the experiment and control colums using an inner join
experiment_data.merge(control_data, how='inner', suffixes=('_click', '_view'), on='id')

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view


The resulting dataframe is empty, meaning that there is no overlap between the contro and experiment groups.

### Now we check to see if anyone clicked but did not view

In [13]:
# We create columns experiment_view_data, experiment_click_data, control_click_data and control_view_data
experiment_click_data = experiment_data[experiment_data.action == 'click']
experiment_view_data = experiment_data[experiment_data.action == 'view']
control_click_data = control_data[control_data.action == 'click']
control_view_data = control_data[control_data.action == 'view']

In [14]:
# we merge the experiment_click_data with experiment_view_data
# We use the left join
#The left join returns all records that match in both tables
experiment_click_members = experiment_click_data.merge(experiment_view_data, on='id', suffixes=('_click', '_view'), how='left')

In [15]:
experiment_click_members

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view
0,2016-09-24 20:58:01.948663,349125,experiment,click,2016-09-24 20:57:20.336757,experiment,view
1,2016-09-24 21:06:27.553057,601714,experiment,click,2016-09-24 21:05:15.348935,experiment,view
2,2016-09-24 21:30:02.739756,487634,experiment,click,2016-09-24 21:29:19.766467,experiment,view
3,2016-09-24 23:01:12.108316,468601,experiment,click,2016-09-24 23:01:08.713402,experiment,view
4,2016-09-25 00:01:47.933853,555973,experiment,click,2016-09-25 00:00:47.700734,experiment,view
...,...,...,...,...,...,...,...
923,2017-01-17 09:27:37.808307,891635,experiment,click,2017-01-17 09:26:26.463667,experiment,view
924,2017-01-17 15:09:17.533736,865656,experiment,click,2017-01-17 15:08:41.151181,experiment,view
925,2017-01-17 20:14:12.483794,413359,experiment,click,2017-01-17 20:14:00.072907,experiment,view
926,2017-01-18 00:56:24.554729,344770,experiment,click,2017-01-18 00:55:55.026210,experiment,view


In [16]:
experiment_click_members.count()

timestamp_click    928
id                 928
group_click        928
action_click       928
timestamp_view     928
group_view         928
action_view        928
dtype: int64

From the experiment, 928 clicked but did not view

In [17]:
# Now we merge the control_click_data with control_view_data
# We use the left join
#The left join returns all records that match in both tables
control_click_members = control_click_members = control_click_data.merge(control_view_data, on='id', suffixes=('_click', '_view'), how='left')

In [18]:
control_click_members

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view
0,2016-09-25 02:53:25.459874,398892,control,click,2016-09-25 02:52:43.844199,control,view
1,2016-09-25 05:19:15.810727,544571,control,click,2016-09-25 05:18:58.565357,control,view
2,2016-09-25 08:25:32.821891,194950,control,click,2016-09-25 08:24:31.192802,control,view
3,2016-09-25 09:45:12.114972,894454,control,click,2016-09-25 09:43:32.734737,control,view
4,2016-09-25 10:38:53.299877,639852,control,click,2016-09-25 10:37:38.286145,control,view
...,...,...,...,...,...,...,...
927,2017-01-17 18:32:30.832981,762498,control,click,2017-01-17 18:32:04.305072,control,view
928,2017-01-17 22:40:54.304413,591686,control,click,2017-01-17 22:39:20.924266,control,view
929,2017-01-17 23:20:35.483601,451198,control,click,2017-01-17 23:19:39.649126,control,view
930,2017-01-17 23:47:58.209653,252195,control,click,2017-01-17 23:46:19.329053,control,view


Out of the control group, 932 clicked but did not view

In [24]:
control_click_members.head()

,timestamp_click,id,group_click,action_click,timestamp_view,group_view,action_view
0,2016-09-25 02:53:25.459874,398892,control,click,2016-09-25 02:52:43.844199,control,view
1,2016-09-25 05:19:15.810727,544571,control,click,2016-09-25 05:18:58.565357,control,view
2,2016-09-25 08:25:32.821891,194950,control,click,2016-09-25 08:24:31.192802,control,view
3,2016-09-25 09:45:12.114972,894454,control,click,2016-09-25 09:43:32.734737,control,view
4,2016-09-25 10:38:53.299877,639852,control,click,2016-09-25 10:37:38.286145,control,view


In [19]:
#Comparing what we have above with the total number of members in the experiment and control group
expview = len(experiment_data[experiment_data.action  == 'view'])
expview

2996

In [20]:
conview = len(control_data[control_data.action == 'view'])
conview

3332

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [21]:
#import relevant libraries for this
import statsmodels.api as sm

In [25]:
zscore, pval = sm.stats.proportions_ztest([control_click_members, experiment_click_members ], [conview,expview ], alternative='smaller')
print(pval, zscore)

ValueError: cannot copy sequence with size 5 to array axis with dimension 7

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [26]:
#Your code here
control_click_rate = control_click_members/conview # calculated above
expected_experiment_clicks = control_click_rate * expview
expected_experiment_clicks

TypeError: unsupported operand type(s) for /: 'str' and 'int'

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.